<a href="https://colab.research.google.com/github/julsCadenas/thesis-chick-detect/blob/main/chicktraining_colab_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Ultralytics and opencv

In [ ]:
!pip install ultralytics

In [ ]:
!pip install opencv-python

import statements
mount your drive and add the directory to your dataset

In [ ]:
from ultralytics import YOLO
import os
import cv2
from google.colab import drive

drive.mount('/content/gdrive')
ROOT_DIR = '/content/gdrive/My Drive/manok/'

load a yolov8 pretrained model

In [ ]:
model = YOLO(os.path.join(ROOT_DIR, "last.pt"))

train the model

In [ ]:
results = model.train(data=os.path.join(ROOT_DIR, "dataset.yaml"), epochs=32)

In [ ]:
!scp -r /content/runs '/content/gdrive/My Drive/manok'

test the model using a video

In [ ]:
# Define paths
VIDEOS_DIR = '/content/gdrive/My Drive/manok/dataset2'
video_path = os.path.join(VIDEOS_DIR, 'chickentry.mp4')
video_path_out = os.path.join(VIDEOS_DIR, 'chickentry_out.mp4')

# Load video
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

if ret:
    H, W, _ = frame.shape
    out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

    model_path = '/content/gdrive/My Drive/manok/runs/detect/train6/weights/last.pt'
    model = YOLO(model_path)

    threshold = 0.5

    while ret:
        results = model(frame)[0]

        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result

            if score > threshold:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
                cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

        out.write(frame)
        ret, frame = cap.read()

    cap.release()
    out.release()
    cv2.destroyAllWindows()
else:
    print("Failed to read the video file.")
